<a href="https://colab.research.google.com/github/FlyAIBox/Agent_In_Action/blob/main/01-agent-llm-mcp/prompt-enginner/AgentWithTools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 智能体工具调用系统 - 详细说明

## 项目概述
这是一个基于大语言模型（LLM）的智能体系统，演示了如何让AI智能体使用工具来执行具体任务。该系统展示了现代AI智能体的核心概念：**工具调用（Tool Calling）**。

## 核心功能
1. **智能体循环（Agent Loop）**：智能体能够持续思考和行动，直到完成任务
2. **工具调用机制**：智能体可以调用预定义的工具（如文件操作）
3. **结构化响应解析**：将LLM的自然语言响应解析为结构化的工具调用指令
4. **记忆管理**：维护对话历史，让智能体能够基于之前的交互做出决策

## 技术架构
- **LLM引擎**：使用LiteLLM调用OpenAI GPT-4o模型
- **工具系统**：预定义的工具函数（list_files, read_file, terminate）
- **解析器**：将LLM响应解析为JSON格式的工具调用
- **循环控制**：防止无限循环的安全机制

## 学习价值
这个示例非常适合我们来理解：
- 智能体如何与外部工具交互
- LLM如何生成结构化的工具调用指令
- 如何构建一个完整的智能体工作流
- 现代AI应用的基本架构模式


In [1]:
# 安装必要的依赖包
!!pip install openai==1.107.0

['Collecting openai==1.107.0',
 '  Downloading openai-1.107.0-py3-none-any.whl.metadata (29 kB)',
 'Requirement already satisfied: anyio<5,>=3.5.0 in /usr/local/lib/python3.12/dist-packages (from openai==1.107.0) (4.11.0)',
 'Requirement already satisfied: distro<2,>=1.7.0 in /usr/local/lib/python3.12/dist-packages (from openai==1.107.0) (1.9.0)',
 'Requirement already satisfied: httpx<1,>=0.23.0 in /usr/local/lib/python3.12/dist-packages (from openai==1.107.0) (0.28.1)',
 'Requirement already satisfied: jiter<1,>=0.4.0 in /usr/local/lib/python3.12/dist-packages (from openai==1.107.0) (0.11.1)',
 'Requirement already satisfied: pydantic<3,>=1.9.0 in /usr/local/lib/python3.12/dist-packages (from openai==1.107.0) (2.11.10)',
 'Requirement already satisfied: sniffio in /usr/local/lib/python3.12/dist-packages (from openai==1.107.0) (1.3.1)',
 'Requirement already satisfied: tqdm>4 in /usr/local/lib/python3.12/dist-packages (from openai==1.107.0) (4.67.1)',
 'Requirement already satisfied: 

In [2]:
# 导入必要的模块
import os, getpass

def _set_env(var: str):
    """
    设置环境变量的辅助函数

    参数:
        var (str): 要设置的环境变量名称

    功能:
        - 检查环境变量是否已存在
        - 如果不存在，则提示用户输入并设置
    """
    if not os.environ.get(var):  # 检查环境变量是否已设置
        os.environ[var] = getpass.getpass(f"{var}: ")  # 安全地获取用户输入

# 设置 OpenAI API 密钥
# 这是使用 OpenAI 模型所必需的
_set_env("OPENAI_API_KEY")
# 设置 OpenAI API代理地址 (例如：https://api.apiyi.com/v1）
_set_env("OPENAI_BASE_URL")

OPENAI_API_KEY: ··········
OPENAI_BASE_URL: ··········


In [3]:
# ===== 导入必要的库 =====
import json  # 用于JSON数据处理
import os    # 用于操作系统相关操作
import sys   # 用于系统相关操作
from typing import List, Dict   # 类型提示，提高代码可读性
from openai import OpenAI       # 用于调用OpenAI API
# ===== 核心工具函数定义 =====

def extract_markdown_block(response: str, block_type: str = "json") -> str:
    """
    从LLM响应中提取代码块内容

    参数:
        response: LLM的原始响应文本
        block_type: 要提取的代码块类型，默认为"json"

    返回:
        提取出的代码块内容
    """
    # 检查响应中是否包含代码块标记
    if not '```' in response:
        return response

    # 分割响应并提取第一个代码块
    code_block = response.split('```')[1].strip()

    # 如果代码块以指定类型开头，则移除类型标识
    if code_block.startswith(block_type):
        code_block = code_block[len(block_type):].strip()

    return code_block

def generate_response(messages: List[Dict]) -> str:
    """
    调用LLM生成响应

    参数:
        messages: 消息列表，包含系统提示和对话历史

    返回:
        LLM生成的响应文本
    """
    # 使用LiteLLM调用OpenAI GPT-4o模型

    client=OpenAI(
        base_url=os.environ['OPENAI_BASE_URL'],
        api_key=os.environ['OPENAI_API_KEY']
    )
    response = client.chat.completions.create(
        model="gpt-4o",  # 指定使用的模型
        messages=messages,      # 传入消息列表
        max_tokens=1024         # 限制最大token数量
    )

    return response.choices[0].message.content.strip()

def parse_action(response: str) -> Dict:
    """
    解析LLM响应，提取结构化的工具调用指令

    参数:
        response: LLM的响应文本

    返回:
        包含工具名称和参数的字典
    """
    try:
        # 从响应中提取action代码块
        response = extract_markdown_block(response, "action")

        # 将JSON字符串解析为Python字典
        response_json = json.loads(response)

        # 验证响应格式是否正确
        if "tool_name" in response_json and "args" in response_json:
            return response_json
        else:
            # 如果格式不正确，返回错误信息
            return {"tool_name": "error", "args": {"message": "You must respond with a JSON tool invocation."}}
    except json.JSONDecodeError:
        # 如果JSON解析失败，返回错误信息
        return {"tool_name": "error", "args": {"message": "Invalid JSON response. You must respond with a JSON tool invocation."}}

# ===== 智能体可用的工具函数 =====

def list_files() -> List[str]:
    """
    列出当前目录中的所有文件

    返回:
        文件名列表
    """
    return os.listdir(".")

def read_file(file_name: str) -> str:
    """
    读取指定文件的内容

    参数:
        file_name: 要读取的文件名

    返回:
        文件内容或错误信息
    """
    try:
        with open(file_name, "r") as file:
            return file.read()
    except FileNotFoundError:
        return f"Error: {file_name} not found."
    except Exception as e:
        return f"Error: {str(e)}"

# ===== 智能体系统提示词定义 =====
# 这个提示词定义了智能体的行为规则和可用工具
agent_rules = [{
    "role": "system",
    "content": """
你是一个AI智能体，可以通过使用可用工具来执行任务。

可用工具:

```json
{
    "list_files": {
        "description": "列出当前目录中的所有文件。",
        "parameters": {}
    },
    "read_file": {
        "description": "读取文件的内容。",
        "parameters": {
            "file_name": {
                "type": "string",
                "description": "要读取的文件名。"
            }
        }
    },
    "terminate": {
        "description": "结束智能体循环并提供任务摘要。",
        "parameters": {
            "message": {
                "type": "string",
                "description": "返回给用户的摘要消息。"
            }
        }
    }
}
```

如果用户询问文件、文档或内容，请先列出文件，然后再读取它们。

当你完成任务后，使用"terminate"工具结束对话，我将向用户提供结果。

重要！！！每个响应都必须包含一个动作。
你必须始终按照以下格式响应：

<停下来逐步思考。参数映射到args。在这里插入你逐步思考的丰富描述。>

```action
{
    "tool_name": "插入工具名称",
    "args": {...在这里填入任何必需的参数...}
}```
"""
}]

# ===== 智能体主循环初始化 =====
iterations = 0        # 当前迭代次数
max_iterations = 10   # 最大迭代次数，防止无限循环

# 获取用户任务
user_task = input("What would you like me to do? ")

# 初始化对话记忆，包含用户的任务
memory = [{"role": "user", "content": user_task}]

# ===== 智能体主循环 =====
# 这是智能体的核心工作循环，持续执行直到任务完成或达到最大迭代次数
while iterations < max_iterations:
    # 1. 构建提示：将智能体规则与对话记忆结合
    prompt = agent_rules + memory

    # 2. 调用LLM生成响应
    print("Agent thinking...")
    response = generate_response(prompt)
    print(f"Agent response: {response}")

    # 3. 解析响应以确定要执行的动作
    action = parse_action(response)
    result = "Action executed"  # 默认结果

    # 4. 根据解析出的动作执行相应的工具函数
    if action["tool_name"] == "list_files":
        result = {"result": list_files()}
    elif action["tool_name"] == "read_file":
        result = {"result": read_file(action["args"]["file_name"])}
    elif action["tool_name"] == "error":
        result = {"error": action["args"]["message"]}
    elif action["tool_name"] == "terminate":
        print(action["args"]["message"])
        break  # 终止循环
    else:
        result = {"error": "Unknown action: " + action["tool_name"]}

    print(f"Action result: {result}")

    # 5. 更新对话记忆，添加智能体响应和执行结果
    memory.extend([
        {"role": "assistant", "content": response},
        {"role": "user", "content": json.dumps(result)}
    ])

    # 6. 检查终止条件
    if action["tool_name"] == "terminate":
        break

    iterations += 1  # 增加迭代计数


What would you like me to do? 告诉我当前目录里有什么内容
Agent thinking...
Agent response: 为了提供准确的信息，我将列出当前目录中的所有文件。
Action result: {'error': 'Invalid JSON response. You must respond with a JSON tool invocation.'}
Agent thinking...
Agent response: ```action
{
    "tool_name": "list_files",
    "args": {}
}
```
Action result: {'result': ['.config', 'sample_data']}
Agent thinking...
Agent response: 当前目录中包含两个项目：一个名为“.config”的项目和另一个名为“sample_data”的项目。如果你需要进一步的信息或需要查看其中的某个内容，请告诉我。我可以帮助读取或探索这些项目内的内容。
Action result: {'error': 'Invalid JSON response. You must respond with a JSON tool invocation.'}
Agent thinking...
Agent response: ```action
{
    "tool_name": "terminate",
    "args": {
        "message": "当前目录中包含两个项目：一个名为“.config”的项目和另一个名为“sample_data”的项目。"
    }
}
```
当前目录中包含两个项目：一个名为“.config”的项目和另一个名为“sample_data”的项目。
